# Carbonate Isotope Analysis
This notebook processes the $\mathrm{\delta^{13}C}$ (`d13C`) and $\mathrm{\delta^{18}O}$ (`d18O`) data from the mass spectrometer measurements of the carbonate fraction (March 2022). These are already corrected to the Carrara Z standard, but the variance in measurements of this standard is used to calculate the error in the isotope measurements.

I also attempt to reconstruct the carbonate content of the samples from the peak height in the mass spec.

In [34]:
# Import libraries
import numpy as np
import pandas as pd
import ipywidgets as wdg
from scipy.stats import linregress
from scipy.optimize import curve_fit
from plotly_default import go, graph_config, sel_trace# Custom graph style

In [2]:
# True delta values of standards
true_std = {'CARRARA Z': {'d18O_carb': -1.27, 'd13C_carb': 2.25},
            'FLETTON CLAY' : {'d18O_carb': -0.3, 'd13C_carb': 2.24}}

standards = list(true_std.keys()) # list of standard names

In [9]:
# Read in MS data
isotope_df = pd.read_csv('./Data/carbonate_isotopes.csv')
isotope_df.head(20)

,sample,mass,est_carbonate,d18O_carb,d13C_carb,carb_mV,carb_diluted
0,CARRARA Z,0.000136,NaN,-1.35,2.22,2605,NaN
1,CARRARA Z,0.000238,NaN,-1.29,2.18,4583,NaN
2,CARRARA Z,0.000224,NaN,-1.24,2.30,4475,NaN
3,FLETTON CLAY,0.000165,NaN,-0.32,2.22,3334,NaN
4,KC-2-0,0.000500,30.0,-5.14,-4.51,7974,NaN
5,KC-2-1,0.001100,18.0,-4.57,-4.53,3883,True
6,KC-2-2,0.020500,1.0,-3.82,-1.28,2844,NaN
7,KC-2-2.5,0.000500,28.0,-4.21,1.24,3643,NaN
8,KC-2-3,0.002600,7.0,-3.56,0.12,2805,NaN
9,KC-2-3.5,0.014200,1.0,-3.95,-0.21,4056,NaN


In [22]:
# Just the Carrara standards
carrara_df = isotope_df[isotope_df['sample'] == 'CARRARA Z']
carrara_df.describe()

,mass,est_carbonate,d18O_carb,d13C_carb,carb_mV
count,20.000000,0.0,20.000000,20.00000,20.000000
mean,0.000177,NaN,-1.274000,2.25100,3563.900000
std,0.000039,NaN,0.041218,0.06155,843.368428
min,0.000126,NaN,-1.350000,2.09000,2415.000000
25%,0.000144,NaN,-1.300000,2.21750,2926.000000
50%,0.000171,NaN,-1.270000,2.27000,3475.500000
75%,0.000212,NaN,-1.250000,2.30000,4303.750000
max,0.000252,NaN,-1.160000,2.32000,5165.000000


In [17]:
# Calculate errors in isotope measurements

d18O_err = carrara_df.d18O_carb.std()
d13C_err = carrara_df.d13C_carb.std()
d13C_err

0.061550150711688525

In [23]:
# Check for systematic trends
xs = list(range(len(carrara_df)))
carrara_check = go.Figure(data=[
    go.Scatter(x=xs, y=carrara_df.d18O_carb, mode='markers', error_y=dict(type='constant', value=d18O_err), name='δ18O'),
    go.Scatter(x=xs, y=carrara_df.d13C_carb, mode='markers', error_y=dict(type='constant', value=d13C_err), name='δ13C'),
])

carrara_check.show()

In [25]:
# update isotope df with errors
isotope_df['d18O_carb_err'] = d18O_err
isotope_df['d13C_carb_err'] = d13C_err
# isotope_df.head()

## Percentage Carbonate Calculation

In [37]:
# Check the peak height correlates with amount of carbonate

regression_out = linregress(carrara_df.mass,carrara_df.carb_mV)

xs = np.linspace(0, 260e-6)
ys = regression_out.slope * xs + regression_out.intercept

def corr_fn(x, m):
    return m*x

mV_slope = curve_fit(corr_fn, carrara_df.mass, carrara_df.carb_mV)[0][0]

yy = mV_slope*xs

car_carb_plot = go.Figure(data=[
    go.Scatter(x=carrara_df.mass, y=carrara_df.carb_mV, mode='markers'),
    go.Scatter(x=xs, y=ys),
    go.Scatter(x=xs, y=yy)
])

car_carb_plot.update_layout(xaxis_title='mass / g', yaxis_title='Peak height / mV',
                            xaxis_range=[0,260e-6], yaxis_range=[0,6000])

car_carb_plot.show()

In [41]:
fletton_df = isotope_df[isotope_df['sample'] == 'FLETTON CLAY']

car_carb_plot.add_trace(go.Scatter(x=fletton_df.mass, y=fletton_df.carb_mV, mode='markers'))

In [44]:
def calculate_carbonate(mV, mass, diluted, dilution_ratio=3):
    if diluted == True:
        mass = mass/dilution_ratio
    else:
        pass

    mass_carb = mV/mV_slope
    
    return 100*mass_carb/mass


carbonate_contents = []

for r in isotope_df.iterrows():
    carbonate_contents.append(calculate_carbonate(r[1].carb_mV, r[1].mass, r[1].carb_diluted))
    
# carbonate_contents

In [59]:
carb_plot = go.FigureWidget()
carb_plot.add_trace(go.Scatter(x=isotope_df.est_carbonate, text=isotope_df['sample'], mode='markers'))
carb_plot.update_layout(xaxis_title='Estimated Carbonate (wt%)', yaxis_title='Calculated Carbonate (wt%)');

In [58]:
@wdg.interact(dilution_ratio=wdg.FloatSlider(value=3., min=1, max=10))
def update_carb_plot(dilution_ratio):
    carbonate_contents = []

    for r in isotope_df.iterrows():
        carbonate_contents.append(calculate_carbonate(r[1].carb_mV, r[1].mass, r[1].carb_diluted, dilution_ratio))
        
    with carb_plot.batch_update():
        carb_plot.select_traces().__next__().y=carbonate_contents

carb_plot

interactive(children=(FloatSlider(value=3.0, description='dilution_ratio', max=10.0, min=1.0), Output()), _dom…

FigureWidget({
    'data': [{'mode': 'markers',
              'text': array(['CARRARA Z', 'CARRARA Z', 'CARRAR…

In [60]:
# Add calculations back into isotope dataframe
isotope_df['calculated_carbonate'] = carbonate_contents

In [61]:
isotope_df

,sample,mass,est_carbonate,d18O_carb,d13C_carb,carb_mV,carb_diluted,d18O_carb_err,d13C_carb_err,calculated_carbonate
0,CARRARA Z,0.000136,NaN,-1.35,2.22,2605,NaN,0.041218,0.06155,94.845219
1,CARRARA Z,0.000238,NaN,-1.29,2.18,4583,NaN,0.041218,0.06155,95.349743
2,CARRARA Z,0.000224,NaN,-1.24,2.30,4475,NaN,0.041218,0.06155,98.921717
3,FLETTON CLAY,0.000165,NaN,-0.32,2.22,3334,NaN,0.041218,0.06155,100.052577
4,KC-2-0,0.000500,30.0,-5.14,-4.51,7974,NaN,0.041218,0.06155,78.968312
...,...,...,...,...,...,...,...,...,...,...
80,KC-3-K-L,0.000800,NaN,-3.72,-1.10,3006,NaN,0.041218,0.06155,18.605683
81,FLETTON CLAY,0.000219,NaN,-0.18,2.31,4369,NaN,0.041218,0.06155,98.783533
82,CARRARA Z,0.000139,NaN,-1.23,2.27,2854,NaN,0.041218,0.06155,101.668355
83,CARRARA Z,0.000170,NaN,-1.25,2.32,3469,NaN,0.041218,0.06155,101.042016


In [77]:
# Reduce the dataframe for export
excluded_samples = [sample for sample in isotope_df['sample'] if sample.endswith('-L')]
excluded_samples += ['CARRARA Z', 'FLETTON CLAY']

red_df = isotope_df[~isotope_df['sample'].isin(excluded_samples)]

red_df = red_df.drop(columns=['mass', 'est_carbonate', 'carb_mV', 'carb_diluted'])

In [78]:
red_df

,sample,d18O_carb,d13C_carb,d18O_carb_err,d13C_carb_err,calculated_carbonate
4,KC-2-0,-5.14,-4.51,0.041218,0.06155,78.968312
5,KC-2-1,-4.57,-4.53,0.041218,0.06155,52.437573
6,KC-2-2,-3.82,-1.28,0.041218,0.06155,0.686946
7,KC-2-2.5,-4.21,1.24,0.041218,0.06155,36.077447
8,KC-2-3,-3.56,0.12,0.041218,0.06155,5.342028
9,KC-2-3.5,-3.95,-0.21,0.041218,0.06155,1.414348
10,KC-2-4.5,-3.64,-0.71,0.041218,0.06155,24.267401
11,KC-2-5,-3.16,-1.42,0.041218,0.06155,2.325173
12,KC-2-6,-3.90,0.77,0.041218,0.06155,33.654458
13,KC-2-6p,-3.50,0.39,0.041218,0.06155,30.586816


In [79]:
red_df.to_csv('Data/carbonates_processed.csv')